# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load
file_to_load = "WeatherData.csv"

# Read Purchasing File and store into Pandas data frame
vacation_df = pd.read_csv(file_to_load)
vacation_df.dropna()
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,0,Alamosa,1,US,1579549978,53,37.47,-105.87,36.00,1.36
1,1,Mahon,75,ES,1579549897,93,39.89,4.27,55.40,18.34
2,2,Umm Lajj,0,SA,1579549978,57,25.02,37.27,58.69,6.85
3,3,Marzuq,0,YE,1579549979,57,14.40,46.47,56.25,2.21
4,4,Bluff,100,NZ,1579549979,96,-46.60,168.33,57.00,1.01
...,...,...,...,...,...,...,...,...,...,...
549,549,Tahe,31,CN,1579549785,96,52.33,124.73,-12.15,2.37
550,550,Nhulunbuy,100,AU,1579550029,78,-12.23,136.77,80.60,4.70
551,551,Ankang,100,CN,1579550029,78,32.68,109.02,37.49,1.39
552,552,Arrondissement de Bastia,90,FR,1579550029,71,42.67,9.33,51.80,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = vacation_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
criteria_df = vacation_df[vacation_df['Max_Temp'].between(70, 80)]
criteria_df = criteria_df[criteria_df['Wind_Speed']<10]
criteria_df = criteria_df = criteria_df[criteria_df['Cloudiness'] == 0]
criteria_df = criteria_df[criteria_df['Max_Temp']<78.80]
criteria_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
158,158,Sao Filipe,0,CV,1579549992,75,14.90,-24.50,71.20,7.87
164,164,De Aar,0,ZA,1579549992,16,-30.65,24.01,76.57,8.14
171,171,Saldanha,0,ZA,1579549774,50,-33.01,17.94,77.00,8.05
197,197,Poya,0,NC,1579549997,72,-21.35,165.15,75.83,6.64
338,338,Pathein,0,MM,1579550010,75,16.78,94.73,74.28,7.38
433,433,Jalingo,0,NG,1579550018,22,8.88,11.37,77.86,5.39


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [6]:
hotel_df = criteria_df
                                 
hotel_df['Hotel Name'] = ""

hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name
158,158,Sao Filipe,0,CV,1579549992,75,14.90,-24.50,71.20,7.87,
164,164,De Aar,0,ZA,1579549992,16,-30.65,24.01,76.57,8.14,
171,171,Saldanha,0,ZA,1579549774,50,-33.01,17.94,77.00,8.05,
197,197,Poya,0,NC,1579549997,72,-21.35,165.15,75.83,6.64,
338,338,Pathein,0,MM,1579550010,75,16.78,94.73,74.28,7.38,
433,433,Jalingo,0,NG,1579550018,22,8.88,11.37,77.86,5.39,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 7000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng to identify Hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel_Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.


In [9]:
narrowed_city_df = hotel_df
narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name,Hotel_Name
158,158,Sao Filipe,0,CV,1579549992,75,14.90,-24.50,71.20,7.87,,Hotel Xaguate
164,164,De Aar,0,ZA,1579549992,16,-30.65,24.01,76.57,8.14,,Boutique Guesthouse De Aar
171,171,Saldanha,0,ZA,1579549774,50,-33.01,17.94,77.00,8.05,,Protea Hotel by Marriott Saldanha Bay
197,197,Poya,0,NC,1579549997,72,-21.35,165.15,75.83,6.64,,NaN
338,338,Pathein,0,MM,1579550010,75,16.78,94.73,74.28,7.38,,The First Hotel
433,433,Jalingo,0,NG,1579550018,22,8.88,11.37,77.86,5.39,,"Fast Track Hotel, Jalingo"


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
marker_locations = narrowed_city_df[['Lat', 'Lng']]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)

fig.add_layer(markers)
fig.add_layer(heat_layer)

fig

# Display Map

Figure(layout=FigureLayout(height='420px'))